In [2]:
# ModelCheckpoint 와 EarlyStopping 콜랩
import keras

callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor='val_acc',
        patience=1, # 1 에포크보다 더 길게 (즉 2 에포크 동안) 정확도가 향상되지 않으면 훈련이 중지된다
    ),
    keras.callbacks.ModelCheckpoint( # 에포크마다 현재 가중치를 저장
        filepath='my_model.h5',
        # val_loss 가 좋아지지 않으면 모델 파일을 덮어쓰지 않는다는 뜻. 훈련하는 동안 가장 좋은 모델을 저장
        monitor='val_loss',
        save_best_only=True,
    )
]

model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['acc']) # 정확도를 모니터링하므로 모델 지표에 포함되어야 한다.

# 콜백이 검증 손실과 검증 정확도를 모니터링하기 때문에 validation_data 매개변수에 검증 데이터를 전달해야한다.
model.fit(x, y,
         epochs=10,
         batch_size=32,
         callbacks=callbacks_list,
         validation_data=(x_val, y_val))

NameError: name 'model' is not defined

In [ ]:
# ReduceLROnPlateau 콜백
# 이 콜백을 사용하면 검증 손실이 향상되지 않을 때 학습률을 작게 할 수 있다. 
# 손실 곡선이 평탄할 때 학습률을 작게 하거나 크게 하면 훈련 도중 지역 최솟값에서 효과적으로 빠져나올 수 있다.
callbacks_list = [
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.1, # 콜백이 호출될 때 학습률을 10배로 줄인다
        patience=10, # 검증 손실이 10 에포크 동안 좋아지지 않으면 콜백이 호출
    )
]

model.fit(x, y,
         epochs=10,
         batch_size=32,
         callbacks=callbacks_list,
         validation_data=(x_val, y_val))

In [ ]:
# 자신만의 콜백 만들기
# 매 에포크의 끝에서 검증 세트의 첫 번째 샘플로 모델에 있는 모든 층의 활성화 출력을 계산하여 (넘파이 배열로) 디스크에 저장하는 자작 콜백의 예
import keras
import numpy as np

class ActivationLogger(keras.callbacks.Callback):
    def set_model(self, model): # 호출하는 모델에 대한 정보를 전달하기 위해 훈련하기 전에 호출
        self.model = model
        layer_outputs = [layers.output for layer in model.layers]
        self.activations_model = keras.models.Model(model.input, layer_outputs) # 각 층의 활성화 출력을 반환하는 Model 객체
        
    def on_epoch_end(self, epoch, logs=None):
        if self.validation_data is None:
            raise RuntimeError('Requires validation_data.')
        
        validation_sample = self.validation_data[0][0:] # 검증 데이터의 첫 번째 샘플을 가져온다
        activations = self.activations_model.predict(validation_sample)
        f = open('activations_at_epoch_' + str(epoch) + '.npz', 'wb')
        np.save(f, activations)
        f.close()